In [1]:
from config import API_TOKEN
import requests
import json

JIRA_URL = "https://Hibernate.atlassian.net/rest/api/3"

USERNAME = "omar.khabthani123@gmail.com"
API_TOKEN = API_TOKEN
def fetch_issues(project_key):
    auth = (USERNAME, API_TOKEN)
    response = requests.get(
        f"{JIRA_URL}/search",
        auth=auth,
        params={"jql": f"project={project_key}"}
    )
    response.raise_for_status()
    return response.json()["issues"]

def fetch_comments(issue_key):
    auth = (USERNAME, API_TOKEN)
    response = requests.get(
        f"{JIRA_URL}/issue/{issue_key}/comment",
        auth=auth
    )
    response.raise_for_status()
    return response.json()["comments"]

def extract_comments_for_project(project_key):
    issues = fetch_issues(project_key)
    
    issue_comments = {}
    for issue in issues:
        comments = fetch_comments(issue["key"])
        issue_comments[issue["key"]] = comments
    
    return issue_comments

def save_to_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)

if __name__ == "__main__":
    project_key = "HHH"
    issue_comments = extract_comments_for_project(project_key)
    json_filename = "issue_comments.json"
    save_to_json(issue_comments, json_filename)
    print(f"Data saved to {json_filename}")

Data saved to issue_comments.json


In [ ]:
#extract comment in html
import requests
from requests.auth import HTTPBasicAuth
import json
from bs4 import BeautifulSoup

url = "https://hibernate.atlassian.net/rest/api/3/issue/HHH-17757/comment?expand=renderedBody"

auth = HTTPBasicAuth("omar.khabthani123@gmail.com",APIKEY)
headers = {
  "Accept": "application/json"
}

response = requests.request(
   "GET",
   url,
   headers=headers,
   auth=auth,
)

comment_data = json.loads(response.text)
comments = comment_data['comments']

for comment in comments:
    rendered_body_html = comment.get('renderedBody', None)
    if rendered_body_html:
        rendered_body_text = BeautifulSoup(rendered_body_html, 'html.parser').get_text()
        print(rendered_body_text)
    else:
        print("No renderedBody found for this comment.")


Related discussion here: https://github.com/jakartaee/persistence/issues/562 


In [ ]:
#extract all comments in plain text
import requests
from requests.auth import HTTPBasicAuth
import json
from bs4 import BeautifulSoup
import pandas as pd

def fetch_issue_keys(project_key):
    url = f"https://hibernate.atlassian.net/rest/api/3/search"
    auth = HTTPBasicAuth("omar.khabthani123@gmail.com",APIKEY)
    headers = {"Accept": "application/json"}
    params = {
        "jql": f"project={project_key}",
        "fields": "key"
    }

    response = requests.get(url, headers=headers, params=params, auth=auth)
    response.raise_for_status()
    data = response.json()

    return [issue["key"] for issue in data.get("issues", [])]

def fetch_comments(issue_keys, url):
    auth = HTTPBasicAuth("omar.khabthani123@gmail.com",APIKEY)
    headers = {"Accept": "application/json"}
    all_comments = []

    for issue_key in issue_keys:
        response = requests.get(url.format(issueIdOrKey=issue_key), headers=headers, auth=auth)
        response.raise_for_status()
        data = response.json()
        for comment in data["comments"]:
            rendered_body = BeautifulSoup(comment.get("renderedBody", ""), 'html.parser').get_text()
            all_comments.append({
                "issue_key": issue_key,
                "comment_id": comment["id"],
                "rendered_body": rendered_body
            })
            
    return all_comments

if __name__ == "__main__":
    project_key = "HHH"
    url = "https://hibernate.atlassian.net/rest/api/3/issue/{issueIdOrKey}/comment?expand=renderedBody"
    
    issue_keys = fetch_issue_keys(project_key)
    issue_comments = fetch_comments(issue_keys, url)
    
    # Convert to DataFrame
    df = pd.DataFrame(issue_comments)
    excel_filename = "jira_comments.xlsx"
    df.to_excel(excel_filename, index=False)
    print(f"DataFrame exported to {excel_filename} successfully.")

DataFrame exported to jira_comments.xlsx successfully.


In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json
from bs4 import BeautifulSoup
import pandas as pd

def fetch_all_project_keys():
    url = "https://hibernate.atlassian.net/rest/api/3/project"
    auth = HTTPBasicAuth("omar.khabthani123@gmail.com",APIKEY)
    headers = {"Accept": "application/json"}
    response = requests.get(url, headers=headers, auth=auth)
    response.raise_for_status()
    data = response.json()
    return [project["key"] for project in data]

def fetch_issue_keys(project_key):
    url = "https://hibernate.atlassian.net/rest/api/3/search"
    auth = HTTPBasicAuth("omar.khabthani123@gmail.com",APIKEY)
    headers = {"Accept": "application/json"}
    params = {
        "jql": f"project={project_key}",
        "fields": "key"
    }
    response = requests.get(url, headers=headers, params=params, auth=auth)
    response.raise_for_status()
    data = response.json()
    return [issue["key"] for issue in data.get("issues", [])]

def fetch_comments(issue_keys, url):
    auth = HTTPBasicAuth("omar.khabthani123@gmail.com",APIKEY)
    headers = {"Accept": "application/json"}
    all_comments = []

    for issue_key in issue_keys:
        response = requests.get(url.format(issueIdOrKey=issue_key), headers=headers, auth=auth)
        response.raise_for_status()
        data = response.json()
        for comment in data["comments"]:
            rendered_body = BeautifulSoup(comment.get("renderedBody", ""), 'html.parser').get_text()
            all_comments.append({
                "issue_key": issue_key,
                "comment_id": comment["id"],
                "rendered_body": rendered_body
            })
            
    return all_comments

if __name__ == "__main__":
    url = "https://hibernate.atlassian.net/rest/api/3/issue/{issueIdOrKey}/comment?expand=renderedBody"
    
    all_project_keys = fetch_all_project_keys()
    all_comments = []

    for project_key in all_project_keys:
        issue_keys = fetch_issue_keys(project_key)
        project_comments = fetch_comments(issue_keys, url)
        all_comments.extend(project_comments)
    
    # Convert to DataFrame
    df = pd.DataFrame(all_comments)
    excel_filename = "jira_comments1.xlsx"
    df.to_excel(excel_filename, index=False)
    print(f"DataFrame exported to {excel_filename} successfully.")

DataFrame exported to jira_comments1.xlsx successfully.


In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json
from bs4 import BeautifulSoup
import pandas as pd

def fetch_all_project_keys():
    url = "https://hibernate.atlassian.net/rest/api/3/project"
    auth = HTTPBasicAuth("omar.khabthani123@gmail.com", APIKEY)
    headers = {"Accept": "application/json"}
    response = requests.get(url, headers=headers, auth=auth)
    response.raise_for_status()
    data = response.json()
    return [{"project_key": project["key"], "project_name": project["name"]} for project in data]

def fetch_issue_keys(project_key):
    url = "https://hibernate.atlassian.net/rest/api/3/search"
    auth = HTTPBasicAuth("omar.khabthani123@gmail.com", APIKEY)
    headers = {"Accept": "application/json"}
    all_issue_keys = []

    start_at = 0
    max_results = 50  
    total = max_results 

    while start_at < total:
        params = {
            "jql": f"project={project_key}",
            "fields": "key",
            "startAt": start_at,
            "maxResults": max_results
        }
        response = requests.get(url, headers=headers, params=params, auth=auth)
        response.raise_for_status()
        data = response.json()

        for issue in data.get("issues", []):
            all_issue_keys.append({"project_key": project_key, "issue_key": issue["key"]})

        start_at += max_results
        total = data.get("total", 0)

    return all_issue_keys

def fetch_comments(issue_keys, url):
    auth = HTTPBasicAuth("omar.khabthani123@gmail.com", APIKEY)
    headers = {"Accept": "application/json"}
    all_comments = []

    for issue_key in issue_keys:
        response = requests.get(url.format(issueIdOrKey=issue_key["issue_key"]), headers=headers, auth=auth)
        response.raise_for_status()
        data = response.json()
        
        comments = data.get("comments")
        if comments:
            for comment in comments:
                rendered_body = BeautifulSoup(comment.get("renderedBody", ""), 'html.parser').get_text()
                all_comments.append({
                    "project_key": issue_key["project_key"],
                    "issue_key": issue_key["issue_key"],
                    "comment_id": comment["id"],
                    "rendered_body": rendered_body
                })
        else:
            all_comments.append({
                "project_key": issue_key["project_key"],
                "issue_key": issue_key["issue_key"],
                "comment_id": None,
                "rendered_body": None
            })
            
    return all_comments

if __name__ == "__main__":
    url = "https://hibernate.atlassian.net/rest/api/3/issue/{issueIdOrKey}/comment?expand=renderedBody"
    
    all_project_keys = fetch_all_project_keys()
    all_comments = []

    for project in all_project_keys:
        issue_keys = fetch_issue_keys(project["project_key"])
        project_comments = fetch_comments(issue_keys, url)
        all_comments.extend(project_comments)

    df = pd.DataFrame(all_comments)
    excel_filename = "jira_comments2.xlsx"
    df.to_excel(excel_filename, index=False)
    print(f"DataFrame exported to {excel_filename} successfully.")